## Mount Google Drive in Colab

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Perform imports

In [0]:
import numpy as np
%tensorflow_version 1.x
import keras
from keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16, preprocess_input as ppi_vgg16
import json

TensorFlow 1.x selected.


Using TensorFlow backend.


## Load balanced dataset

In [0]:
data = np.load('/content/drive/My Drive/Final Capstone/Balanced Data.npy')

## Separate image data from label data

In [0]:
# image data
X = data[:, :-1]
# label data
Y = data[:, -1:]
del data

## Reshape image and label data and create one-hot encoding for label data

In [0]:
# reshape image data to 50 x 50 x 3 format
X = np.reshape(X, (X.shape[0], 50, 50, 3), order='C')
# reshape label data to 1-D array
Y = np.reshape(Y, (Y.shape[0]))
# create one-hot encoding for label data
Y = keras.utils.to_categorical(Y, 2, 'float32')

## Set common hyperparameters

In [0]:
# fraction of data to use for validation--taken from the bottom of the dataset by default in Keras
val_split = 0.2
# number of filters used in the Conv2D layers
filters = 32
# kernel size used in the Conv2D layers
kernel_size = (3, 3)
# pool size used in MaxPooling2D layers
pool_size = (2, 2)
# dropout rate used in Dropout
dropout_rate = 0.25
# width of hidden layer used prior to prediction dense layer
hidden_dense_size = 64
# initial learning rate used in optimizer
initial_lr = 0.01

## Create training and validation sets for use by model evaluation function

In [0]:
from math import ceil
X_val = X[ceil(X.shape[0] * (1 - val_split)):]
Y_val = Y[ceil(X.shape[0] * (1 - val_split)):]

## Modified optimization function

In [0]:
# function is similar to function of same name in 'Run NN Architectures -- #4' with the modification that it is run for a set # of epochs
# and returns a trained model rather than a dictionary of results
def optimize_model(model, X, Y, val_split, initial_lr, epoch_to_stop):
  epoch_count = 0
  history_dict = None
  sgd = SGD(lr=initial_lr)
  sgd_lr = sgd.get_config()['lr']
  # while loop based on model train accuracy convergence
  while (True):
    # while loop based on learning rate reduction
    while (True):
      model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
      old_weights = model.get_weights()
      history = model.fit(X, Y, epochs=1, validation_split=val_split, verbose=1)

      if (history_dict == None):
        history_dict = history.history
        break
      else:
        last_loss = history_dict['loss'][len(history_dict['loss']) - 1]
        curr_loss = history.history['loss'][0]
        if (curr_loss <= last_loss):
          for key in history_dict.keys():
            history_dict[key].append(history.history[key][0])
          break
        else:
          sgd = SGD(lr=sgd_lr/2)
          sgd_lr = sgd.get_config()['lr']
          model.set_weights(old_weights)

    epoch_count += 1

    if (epoch_count >= epoch_to_stop):
      break
  return model

## Function used to calculate the accuracy, sensitivity, and specificity of a trained model

In [0]:
def evaluate_model(X_val, Y_val, model):
  # make predictions on validation set
  Y_pred = model.predict(X_val)
  # counter for IDC-positive images correctly classified
  pos_correct = 0
  # counter for IDC-negative images correctly classified
  neg_correct = 0
  # counter for IDC-positive images
  pos = 0
  # counter for IDC-negative images
  neg = 0
  # loop through predictions made on validation set
  for i in range(Y_pred.shape[0]):
    # correct classification
    true = np.argmax(Y_val[i])
    # predicted classification
    pred = np.argmax(Y_pred[i])
    # logic for IDC-positive images
    if (true == 1):
      pos += 1
      if (true == pred):
        pos_correct += 1
    # logic for IDC-negative images
    elif (true == 0):
      neg += 1
      if (true == pred):
        neg_correct += 1
  
  # calculate sensitivity
  sensitivity = (pos_correct/pos) * 100
  # calculate specificity
  specificity = (neg_correct/neg) * 100
  # calculate accuracy
  accuracy = (pos_correct + neg_correct) / (pos + neg) * 100
  
  print('Sensitivity:  {}%\nSpecificity:  {}%\nAccuracy:  {}%'.format(sensitivity, specificity, accuracy))

## Load dictionary storing the number of epochs for which to run each architecture

In [0]:
with open('/content/drive/My Drive/Final Capstone/model_epochs.json', 'r') as f:
  epoch_dict = json.load(f)

## Run and evaluate each architecture

In [0]:
# Conv2D, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Flatten(data_format='channels_last')(output_layer1)
output_layer3 = Dense(units=2, activation='softmax')(output_layer2)
model = Model(input_layer, output_layer3)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_f_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  78.26978532521628%
Specificity:  84.13407118601434%
Accuracy:  81.22956803250071%


In [0]:
# Conv2D, MaxPooling, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=2, activation='softmax')(output_layer3)
model = Model(input_layer, output_layer4)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_mp_f_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  83.0567125921179%
Specificity:  79.8138598918375%
Accuracy:  81.42000190433872%


In [0]:
# Conv2D, MaxPooling, Flatten, Dropout, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dropout(rate=dropout_rate)(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_mp_f_dr_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  69.43928228132009%
Specificity:  90.34083763048673%
Accuracy:  79.98857396768972%


In [0]:
# Conv2D, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Flatten(data_format='channels_last')(output_layer1)
output_layer3 = Dense(units=hidden_dense_size, activation='relu')(output_layer2)
output_layer4 = Dense(units=2, activation='softmax')(output_layer3)
model = Model(input_layer, output_layer4)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_f_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  87.42710669657161%
Specificity:  76.80794868569991%
Accuracy:  82.06747706858793%


In [0]:
# Conv2D, MaxPooling, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=hidden_dense_size, activation='relu')(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_mp_f_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  80.0%
Specificity:  84.6874606967677%
Accuracy:  82.36582346780081%


In [0]:
# Conv2D, MaxPooling, Flatten, Dropout, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dropout(rate=dropout_rate)(output_layer3)
output_layer5 = Dense(units=hidden_dense_size, activation='relu')(output_layer4)
output_layer6 = Dense(units=2, activation='softmax')(output_layer5)
model = Model(input_layer, output_layer6)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_mp_f_dr_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  73.00865107337393%
Specificity:  89.0265375424475%
Accuracy:  81.09309042435015%


In [0]:
# Conv2D, Conv2D, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=2, activation='softmax')(output_layer3)
model = Model(input_layer, output_layer4)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_c_f_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  84.90868311438642%
Specificity:  81.46773990692995%
Accuracy:  83.17199352524835%


In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_c_mp_f_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  87.504005126562%
Specificity:  79.35479813859891%
Accuracy:  83.39099247786206%


In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dropout, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dropout(rate=dropout_rate)(output_layer4)
output_layer6 = Dense(units=2, activation='softmax')(output_layer5)
model = Model(input_layer, output_layer6)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_c_mp_f_dr_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  79.12207625760973%
Specificity:  86.42937995220727%
Accuracy:  82.81016916875615%


In [0]:
# Conv2D, Conv2D, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = Flatten(data_format='channels_last')(output_layer2)
output_layer4 = Dense(units=hidden_dense_size, activation='relu')(output_layer3)
output_layer5 = Dense(units=2, activation='softmax')(output_layer4)
model = Model(input_layer, output_layer5)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_c_f_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  66.01089394424864%
Specificity:  92.62985787951202%
Accuracy:  79.44583743295141%


In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dense(units=hidden_dense_size, activation='relu')(output_layer4)
output_layer6 = Dense(units=2, activation='softmax')(output_layer5)
model = Model(input_layer, output_layer6)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_c_mp_f_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  85.69689202178789%
Specificity:  82.77575147780153%
Accuracy:  84.22255371822135%


In [0]:
# Conv2D, Conv2D, MaxPooling, Flatten, Dropout, Dense, Dense

input_layer = Input(shape=X.shape[1:4])
output_layer1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(input_layer)
output_layer2 = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), data_format='channels_last', activation='relu')(output_layer1)
output_layer3 = MaxPooling2D(pool_size=pool_size, strides=(1, 1), data_format='channels_last')(output_layer2)
output_layer4 = Flatten(data_format='channels_last')(output_layer3)
output_layer5 = Dropout(rate=dropout_rate)(output_layer4)
output_layer6 = Dense(units=hidden_dense_size, activation='relu')(output_layer5)
output_layer7 = Dense(units=2, activation='softmax')(output_layer6)
model = Model(input_layer, output_layer7)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['c_c_mp_f_dr_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  83.7936558795258%
Specificity:  85.41692868821532%
Accuracy:  84.61294315548925%


In [0]:
# VGG16 model without pretrained weights and with a dense prediction layer

model_vgg16 = VGG16(include_top=False, weights=None, input_shape=X.shape[1:4], pooling='max')
input_layer = Input(shape=X_vgg16.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=2, activation='softmax')(model_vgg16_layer)
model = Model(input_layer, output_layer)

trained_model = optimize_model(model, X_vgg16, Y, val_split, initial_lr, epoch_dict['vgg16_d_np'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  93.91861582826017%
Specificity:  64.65224500062885%
Accuracy:  79.14749103373853%


In [0]:
# VGG16 model without pretrained weights and with a dense hidden layer and a dense prediction layer

model_vgg16 = VGG16(include_top=False, weights=None, input_shape=X.shape[1:4], pooling='max')
input_layer = Input(shape=X.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=hidden_dense_size, activation='relu')(model_vgg16_layer)
output_layer1 = Dense(units=2, activation='softmax')(output_layer)
model = Model(input_layer, output_layer1)

trained_model = optimize_model(model, X, Y, val_split, initial_lr, epoch_dict['vgg16_d_d_np'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  80.11534764498558%
Specificity:  85.87599044145391%
Accuracy:  83.02282032564192%


## Remove image and label data from memory

In [0]:
del X_val
del Y_val
del X
del Y
del model
del trained_model

## Load balanced dataset and preprocess input for use with VGG16 model

In [0]:
data = np.load('/content/drive/My Drive/Final Capstone/Balanced Data.npy')

In [0]:
X = data[:, :-1]
Y = data[:, -1:]
del data

In [0]:
X = np.reshape(X, (X.shape[0], 50, 50, 3), order='C')
Y = np.reshape(Y, (Y.shape[0]))
Y = keras.utils.to_categorical(Y, 2, 'float32')

In [0]:
X_vgg16 = ppi_vgg16(X * 255)
del X

In [0]:
from math import ceil
X_val = X_vgg16[ceil(X_vgg16.shape[0] * (1 - val_split)):]
Y_val = Y[ceil(X_vgg16.shape[0] * (1 - val_split)):]

In [0]:
# VGG16 model with frozen pretrained weights and a dense prediction layer

model_vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=X_vgg16.shape[1:4], pooling='max')

for layer in model_vgg16.layers:
  layer.trainable = False

input_layer = Input(shape=X_vgg16.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=2, activation='softmax')(model_vgg16_layer)
model = Model(input_layer, output_layer)

trained_model = optimize_model(model, X_vgg16, Y, val_split, initial_lr, epoch_dict['vgg16_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  70.84908683114386%
Specificity:  86.50484215821909%
Accuracy:  78.75075380074269%


In [0]:
# VGG16 with frozen pretrained weights, a hidden dense layer, and a dense prediction layer

model_vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=X_vgg16.shape[1:4], pooling='max')

for layer in model_vgg16.layers:
  layer.trainable = False

input_layer = Input(shape=X_vgg16.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=hidden_dense_size, activation='relu')(model_vgg16_layer)
output_layer1 = Dense(units=2, activation='softmax')(output_layer)
model = Model(input_layer, output_layer1)

trained_model = optimize_model(model, X_vgg16, Y, val_split, initial_lr, epoch_dict['vgg16_d_d'])

In [0]:
evaluate_model(X_val, Y_val, trained_model)

Sensitivity:  77.80198654277474%
Specificity:  84.04603194566721%
Accuracy:  80.9534389183356%


Select Conv2D, Conv2D, MaxPooling, Flatten, Dropout, Dense, Dense model for grid search on hyperparameters due to good specificity and overall accuracy without significant volatility between sensitivity and specificity.

Additionally, a simple convolutional autoencoder will be developed to attempt explicit feature extraction from the dataset.  The encoding part of this autoencoder will then serve as input to a model consisting of a hidden dense layer and a prediction dense layer.  The results of this model will be compared to the model selected above.